In [1]:
### no brand names and search terms vector

In [1]:
%load_ext Cython

In [2]:
%%time
%%cython
from gensim.models import doc2vec
import nltk
import re
def split_sentence (sentence):
    return re.split('\W+',sentence)
    
class MyDocs(object):
    def __iter__(self):
        for i, text in enumerate(open("txt_bd.txt")): # doesn't use brand name
            yield doc2vec.LabeledSentence(words=split_sentence(text), tags=['%s' % i])
            # Train the doc2vec model
cdef desp = MyDocs()
model = doc2vec.Doc2Vec(desp, size = 200, window = 8, min_count = 5, workers = 4)
model.save('distance_bd_nst.model')

CPU times: user 40.2 s, sys: 15.8 s, total: 56 s
Wall time: 44.7 s


In [3]:
model = doc2vec.Doc2Vec.load('distance_bd_nst.model')

In [4]:
import numpy as np
st_arrays = np.zeros((240760, 200))
for i,text in enumerate(open("txt_st.txt")): # input search terms 
    st_arrays[i]=model.infer_vector(split_sentence(text))   
st_arrays

array([[-0.01771345, -0.10843907,  0.00612975, ...,  0.02161694,
        -0.10378201, -0.01844648],
       [-0.01429115, -0.11019114,  0.00556979, ...,  0.02254773,
        -0.10421254, -0.01555203],
       [-0.0178379 , -0.10896894,  0.0033653 , ...,  0.01821378,
        -0.10471214, -0.01828229],
       ..., 
       [-0.01697507, -0.11329293,  0.0042711 , ...,  0.02283123,
        -0.105679  , -0.01896542],
       [-0.01861155, -0.11137127,  0.00362843, ...,  0.02187587,
        -0.1032336 , -0.01890764],
       [-0.01793515, -0.11234355,  0.00329106, ...,  0.02135231,
        -0.10515702, -0.01803632]])

In [5]:
## calculate the cosin distance between 2 vectors to generate the 3rd features
# Compute the cosine similarity values between the input text and all archived reviews
# cossims_with_input = map(lambda v: cossim(input_vec, v), model.docvecs) 
# need to chaneg the code into rows: calculate the cosin distance between the vectors in same rows
import numpy as np
import pandas as pd
# Calculate the cosine similarity between two vecotrs 
def cossim(v1, v2):
    return np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / np.sqrt(np.dot(v2, v2))

cossims_with_st=np.zeros((240760,1))
for i in range(240760):
    cossims_with_st[i] = cossim(st_arrays[i], model.docvecs[i])

In [6]:
np.savetxt("distance_bd_nst.csv",cossims_with_st, delimiter=",")

In [7]:
coswithst=pd.read_csv("distance_bd_nst.csv",header=None)
coswithst


,0
0,-0.365117
1,-0.099227
2,-0.198919
3,0.012654
4,-0.002966
5,-0.017634
6,0.007035
7,0.007629
8,0.565678
9,-0.166749


In [ ]:
cws_trn=coswithst.iloc[:num_train]
cws_tst=coswithst.iloc[num_train:]